In [22]:
from tkinter import *
import os, sys
import time
import tkinter
import os
from tkinter import messagebox
import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
import requests
import pandas as pd

In [37]:
def scrape(num_tweets, keyword, start_date, stop_date, save_location):
    scrolls = int(num_tweets/10) + 30
    url = 'https://twitter.com/search?q='
    url = url + keyword + ' ' + "since%3A" + str(start_date)[:-9] + ' until%3A' + str(stop_date)[:-9]
    #print(url)
    browser = webdriver.Chrome("C:\\Users\\elonm\\chromedriver.exe")
    #browser = webdriver.Chrome()
    browser.get(url)
    time.sleep(4)
    body = browser.find_element_by_tag_name('body')
    for _ in range(scrolls):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
    #tweets = browser.find_elements_by_class_name('tweet-text')
    print("Scraping now!")
    tweets = browser.find_elements_by_xpath('/html/body/div/div/div/div/div/div/div/div/div/div/div/div/ol/li/div/div/div/p[@class="TweetTextSize  js-tweet-text tweet-text"]')
    favorites = browser.find_elements_by_xpath('/html/body/div/div/div/div/div/div/div/div/div/div/div/div/ol/li/div/div/div/div/div[@class="ProfileTweet-action ProfileTweet-action--favorite js-toggleState"]/button[@class="ProfileTweet-actionButton js-actionButton js-actionFavorite"]/span/span')
    comments =  browser.find_elements_by_xpath('/html/body/div/div/div/div/div/div/div/div/div/div/div/div/ol/li/div/div/div/div/div[@class="ProfileTweet-action ProfileTweet-action--reply"]/button[@class="ProfileTweet-actionButton js-actionButton js-actionReply"]/span/span')
    retweets  = browser.find_elements_by_xpath('/html/body/div/div/div/div/div/div/div/div/div/div/div/div/ol/li/div/div/div/div/div[@class="ProfileTweet-action ProfileTweet-action--retweet js-toggleState js-toggleRt"]/button[@class="ProfileTweet-actionButton  js-actionButton js-actionRetweet"]/span/span')
    timestamp = browser.find_elements_by_xpath('/html/body/div/div/div/div/div/div/div/div/div/div/div/div/ol/li/div/div/div/small/a/span[1]')
    print("Scraping Done!")
    print(len(tweets), len(retweets), len(favorites), len(timestamp),  len(comments))
    print("Making Dataframe. . .")
    data = pd.DataFrame()
    text = []
    times = []
    fav = []
    com = []
    retwee = []
    for x in tweets:
        text.append(x.text) 
    for x in timestamp:
        times.append(x.text)
    for x in favorites:
        fav.append(x.text)
    for x in comments:
        com.append(x.text)
    for x in retweets:
        retwee.append(x.text)
    data['tweets'] = text
    data['timestamp'] = times 
    data['favorite'] = fav
    data['reply'] = com
    data['retweet'] = retwee
    print("Finished making dataframe!")
    print(data.shape, data.head())
    data.to_csv(save_location, header=True)

In [34]:
# root = tkinter.Tkinter.Tk()
# root.configure(background='black')
window = Tk()
window.resizable(False, False)
window.title("Mr_TweetALot")
# window.geometry('600x500')
window.config(bg='white')

# num_tweets = ''
# keyword = ''
# start_date = ''
# stop_date = ''
# keyword = ''
# save_location = ''

#All labels go below
lbl1 = Label(window, text="Enter the number of tweets to be scraped:", font=("Arial Bold", 15), bg='white', anchor='ne')
lbl1.grid(column=0, row=3)
lbl2 = Label(window, text="From Date(YYYY-MM-DD):", font=("Arial Bold", 15), bg='white', anchor='ne')
lbl2.grid(column=0, row=4)
lbl3 = Label(window, text="To Date(YYYY-MM-DD):", font=("Arial Bold", 15), bg='white')
lbl3.grid(column=0, row=5)
lbl4 = Label(window, text="Search Keys:", font=("Arial Bold", 15), bg='white')
lbl4.grid(column=0, row=6)
lbl5 = Label(window, text="Save Location:", font=("Arial Bold", 15), bg='white')
lbl5.grid(column=0, row=7)
#Labels end

def clicked():
    num_tweets = txt1.get()
    start_date = txt2.get()
    stop_date = txt3.get()
    keyword = txt4.get()
    save_location = txt5.get()
    if num_tweets=='' or start_date == '' or stop_date == '' or keyword == '' or save_location == '':
        messagebox.showerror("Missing Input", "Please fill in all the fields before scraping.")
    if num_tweets.isnumeric() == False:
        print("catch")
        messagebox.showerror("Invalid Input", "The value of number of tweets should be numeric only.")
    num_tweets = round(int(num_tweets))
    if num_tweets < 1:
        messagebox.showerror("Invalid Input", "The value of number of tweets should be more than 0.")
    start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    stop_date = datetime.datetime.strptime(stop_date, "%Y-%m-%d")
    if start_date > stop_date:
        messagebox.showerror("Value Error", "Start date can't be bigger than stop-date")
    current_date =  datetime.datetime.now()
    if stop_date > current_date:
        messagebox.showerror("Value Error", "Date can't be in the future")
    messagebox.showinfo("", "Scraping now. . .\nYou'll be notified when finished.")
    scrape(num_tweets, keyword, start_date, stop_date, save_location)
    messagebox.showinfo("DONE SCRAPING!", "You data has been stored at the location specified.")
    #lbl1.configure(text= res)

#All buttons go below
btn1 = Button(window, text="About", bg="#1DA1F2", fg="white", font=("Arial Bold", 10), borderwidth=0)
btn1.grid(row = 0, column= 1)
btn2 = Button(window, text="Usage", bg="#1DA1F2", fg="white", font=("Arial Bold", 10), borderwidth=0)
btn2.grid(row = 0, column = 2)
btn = Button(window, text="Begin Scraping!", bg="#1DA1F2", fg="white", command=clicked, font=("Arial Bold", 20), borderwidth=0)
btn.grid(column=0, row=10)
#Buttons end

#Text Boxes 
txt1= Entry(window,width=20)
txt1.grid(column=2, row = 3)
txt2= Entry(window,width=20)
txt2.grid(column=2, row=4)
txt3= Entry(window,width=20)
txt3.grid(column=2, row=5)
txt4= Entry(window,width=20)
txt4.grid(column=2, row=6)
txt5= Entry(window,width=20)
txt5.grid(column=2, row=7)
#Text Boxes end

#Click Events Below

#Click Events End

#Read Input

#Read Input End


#Others

#Others end


window.mainloop()

Scraping now!
Scraping Done!
60 60 60 60 60
Making Dataframe. . .
Finished making dataframe!
(60, 5)                                               tweets timestamp favorite reply  \
0  Common site at all Trump Rally’s is huge long ...     Oct 4      252     6   
1  Kavanaugh said himself he wrote his statement ...     Oct 4      157     5   
2  Republican Senate candidate Karin Housley come...     Oct 4      360    13   
3  Less than five weeks until election day, but t...     Oct 4      194     3   
4  SaltyTears! #ComfirmJudgeKavanaugh #ThursdayTh...     Oct 4       11     3   

  retweet  
0     176  
1      92  
2      85  
3      36  
4       8  
